<a href="https://colab.research.google.com/github/cinqbox/liquipedia-esports/blob/main/liquipedia%E3%80%80%E5%8F%8E%E9%9B%86%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pprint
import time
import re
import pandas as pd

# 結果まとめ関数
def result(pattern, string):
  ans = "".join(re.findall(pattern, tes, flags=re.MULTILINE))
  return ans



# ゲームタイトル
game_title = "arenaofvalor"
S = requests.Session()
url = "https://liquipedia.net/" + game_title + "/api.php"

# 日本大会の取得
params = {"action": "query",
          "format": "json",
          "list": "categorymembers",
          "cmprop": "title",
          "cmlimit": "100",
          "cmtitle": "Category:Japanese Tournaments"}

R = S.get(url=url, params=params)
DATA = R.json()
print(R)
pprint.pprint(DATA)
time.sleep(10)

<Response [200]>
{'batchcomplete': '',
 'query': {'categorymembers': [{'ns': 2,
                                'title': 'User:AIC Qualifying of '
                                         'Japan/2019/2019'},
                               {'ns': 0, 'title': 'Arena of Japan'},
                               {'ns': 0,
                                'title': 'Arena of Japan/2019/Event 8'},
                               {'ns': 0, 'title': 'Arena of Japan/2020/Final'},
                               {'ns': 0,
                                'title': 'Arena of Valor International '
                                         'Championship/2019/Japan'},
                               {'ns': 0,
                                'title': 'Arena of Valor Japan '
                                         'League/2020/Spring'},
                               {'ns': 0,
                                'title': 'Arena of Valor World Cup/2019/Japan'},
                               {'ns': 0,
             

In [ ]:
# タイトル一覧を抽出
title_list = []
title = DATA["query"]["categorymembers"]
for i in title:
  title_list.append(i["title"])
  print(i["title"])

User:AIC Qualifying of Japan/2019/2019
Arena of Japan
Arena of Japan/2019/Event 8
Arena of Japan/2020/Final
Arena of Valor International Championship/2019/Japan
Arena of Valor Japan League/2020/Spring
Arena of Valor World Cup/2019/Japan
Arena of Valor World Cup/2019/Japan/Online Qualifier/1
Arena of Valor World Cup/2019/Japan/Online Qualifier/2


In [ ]:
# 結果まとめシート
result_df = pd.DataFrame()

for title in title_list:
  # データの取得
  params = {"action": "query",
          "format": "json",
          "prop": "revisions",
          "rvprop": "content|ids",
          "titles": title}

  R = S.get(url=url, params=params)
  DATA = R.json()
  ids = list(DATA["query"]["pages"].keys())[0]
  # print(ids)
  # print(R)
  # pprint.pprint(DATA)

  tes = DATA["query"]["pages"][ids]["revisions"][0]['*']

  pattern_prize = r"^\|prizepool=(.*?)$"
  pattern_type = r"^\|type=(.*?)$"
  pattern_country = r"^\|country=(.*?)$"
  pattern_prize_local = r"^\|localcurrency=(.*?)$"
  pattern_team_number = r"^\|team_number=(.*?)$"
  pattern_name = r"^\|name=(.*?)$"
  pattern_prize_usd = r"^\|prizepoolusd=(.*?)$"


  prize_local = result(pattern_prize_local, tes)
  holdtype = result(pattern_type, tes)
  team_number = result(pattern_team_number, tes)
  country = result(pattern_country, tes)
  name = result(pattern_name, tes)

  if prize_local == "":
    prize = result(pattern_prize_usd, tes)
    prize_local = "usd"
  else:
    prize = result(pattern_prize, tes)


  dic = {"大会名": name, "賞金": prize, "通貨": prize_local, "開催方法": holdtype, "チーム数": team_number, "開催地域": country}

  # print(dic)

  # データの整形、結合
  df = pd.DataFrame.from_dict(dic, orient='index').T
  print(df)
  result_df = pd.concat([result_df, df], ignore_index=True)
  
  # 加重アクセス防止
  time.sleep(15)


                            大会名 賞金  ... チーム数   開催地域
0  AIC 2019 Qualifying of JAPAN     ...    6  Japan

[1 rows x 6 columns]
              大会名 賞金   通貨    開催方法 チーム数   開催地域
0  Arena of JAPAN     usd  Online       Japan
                                  大会名 賞金  ...                   チーム数   開催地域
0  Arena of Japan - Round 8 ~Revival~     ...  60|teamfirst=blizzard  Japan

[1 rows x 6 columns]
                               大会名      賞金  ...               チーム数   開催地域
0  Arena of Japan Season 1 - Final  25,000  ...  5|teamfirst=scarz  Japan

[1 rows x 6 columns]
                        大会名 賞金  ...                             チーム数   開催地域
0  AIC 2019 Japan Qualifier     ...  6|teamfirst=unsold stuff gaming  Japan

[1 rows x 6 columns]
                                       大会名 賞金   通貨    開催方法          チーム数   開催地域
0  Arena of Valor Japan League Spring 2020     usd  Online  4|teamfirst=  Japan
                                             大会名 賞金   通貨     開催方法 チーム数   開催地域
0  Arena of Valor World Cu

In [ ]:

display(result_df)

,大会名,賞金,通貨,開催方法,チーム数,開催地域
0,AIC 2019 Qualifying of JAPAN,,usd,'''final:'''</br>Online</br>'''Other than the ...,6,Japan
1,Arena of JAPAN,,usd,Online,,Japan
2,Arena of Japan - Round 8 ~Revival~,,usd,Online,60|teamfirst=blizzard,Japan
3,Arena of Japan Season 1 - Final,"25,000",jpy,Online,5|teamfirst=scarz,Japan
4,AIC 2019 Japan Qualifier,,usd,Offline & Online,6|teamfirst=unsold stuff gaming,Japan
5,Arena of Valor Japan League Spring 2020,,usd,Online,4|teamfirst=,Japan
6,Arena of Valor World Cup 2019 Japan Qualifier,,usd,Offline,4,Japan
7,Arena of Valor World Cup 2019 Japan Online Qua...,,usd,Online,15,Japan
8,Arena of Valor World Cup 2019 Japan Online Qua...,,usd,Online,28,Japan


In [ ]:
import numpy as np
result_df.replace("", np.nan, inplace=True)

In [ ]:
print(len(result_df))

9


In [ ]:
result_df["通貨"] = result_df["通貨"].str.upper()
result_df = result_df.dropna(how="any")
result_df = result_df[~result_df["大会名"].duplicated()]
print(len(result_df))
result_df

1


,大会名,賞金,通貨,開催方法,チーム数,開催地域
3,Arena of Japan Season 1 - Final,"25,000",JPY,Online,5|teamfirst=scarz,Japan


In [ ]:
result_df.to_csv(game_title + ".csv", encoding="utf-8-sig", index=False)